# <font color=#025dfa> Derin Öğrenme Regresyon ile Amazon İçin Önerilen Sistemi - Recommended System for Amazon with Deep Learning Regression

TR = Her yorum satırı kendisini üstündeki koda aittir. İlk olarak Türkçe, son olarak İngilizce yazıldı.

EN = Each comment line belongs to the code above it. It was first written in Turkish and lastly in English.

TR = Bu proje, Amazon platformu için kullanıcıların satın alma davranışlarını analiz ederek kişiselleştirilmiş öneriler sunan bir regresyon modeli geliştirmeyi hedeflemektedir. Kullanıcıların geçmiş alışveriş verileri, ürün özellikleri ve kullanıcı profilleri gibi çeşitli veriler kullanılarak, kullanıcıların ilgisini çekebilecek ürünlerin tahmin edilmesi sağlanacaktır. Derin öğrenme teknikleri ile desteklenen bu öneri sistemi, kullanıcı deneyimini iyileştirmek ve satışları artırmak amacıyla daha etkili ve doğru öneriler sunmayı amaçlamaktadır. Proje, e-ticaret sektöründe rekabet avantajı elde etmek için veri odaklı karar verme süreçlerini güçlendirmeyi hedeflemektedir.

EN = This project aims to develop a regression model that analyzes users' purchasing behaviors for the Amazon platform and provides personalized recommendations. Various data such as users' past shopping data, product features, and user profiles will be used to predict products that may interest users. Supported by deep learning techniques, this recommendation system aims to provide more effective and accurate recommendations to improve user experience and increase sales. The project aims to strengthen data-driven decision-making processes to gain competitive advantage in the e-commerce sector.

Kaynak/Source = https://www.kaggle.com/competitions/recommended-system-for-amazon-icl-2021

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import json
import re
import pickle

import warnings
warnings.filterwarnings('ignore')

from scipy import spatial #harital üzerindeki mesafeyi ölçüyor

from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization,Flatten, LeakyReLU
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler
from scipy import spatial

In [2]:
df=pd.read_csv('train.csv')

In [3]:
print(df.head())

           userId   productId  Rating
0   AOPE42H34R0EC  B00000DM9W     5.0
1  A1GI09JC6L0NF7  B00004SABJ     4.0
2   AZLZII4AFX56R  B00000J579     3.0
3  A34AHNT6GD9FWW  9888002198     5.0
4  A2PXRAO5C1XTLW  0972683275     5.0


In [4]:
print(df.sample(5))

               userId   productId  Rating
17343  A2KFK3WT8VAJC5  B00004SPUN     5.0
20627  A1DQGI584UAKRI  B00001WRSJ     5.0
2410   A3R3A8D3D9JVWU  B00000J3H5     5.0
15640  A228FCKXMDRW59  B00000J434     5.0
7515   A1R7POV8N6O5MZ  B00004R8V6     5.0


In [5]:
print(df.tail())

               userId   productId  Rating
33970  A22GYGQ14GHSCD  B00004TDLD     5.0
33971  A2O30HQWWYD5FH  B000038ABH     5.0
33972  A37OTRJO1NM63H  B00003ETSJ     1.0
33973  A2SLR2VUDUGCQM  B00004TDLD     5.0
33974  A339ZN69W7N8PW  B00001P4XH     4.0


In [6]:
print(df.shape)

(33975, 3)


In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33975 entries, 0 to 33974
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     33975 non-null  object 
 1   productId  33975 non-null  object 
 2   Rating     33975 non-null  float64
dtypes: float64(1), object(2)
memory usage: 796.4+ KB
None


In [8]:
print(df.isnull().sum().sort_values(ascending=True))

userId       0
productId    0
Rating       0
dtype: int64


In [9]:
df.iloc[195]

userId       A38YWVKHDGWXFF
productId        B00004T8R2
Rating                  5.0
Name: 195, dtype: object

In [12]:
df.iloc[241]

userId       A3B2X7BT9UCAR3
productId        B00000J40W
Rating                  4.0
Name: 241, dtype: object

In [13]:
df['Rating']=df['Rating'].astype(int)

In [14]:
df.Rating.value_counts()

Rating
5    19004
4     6958
1     3492
3     2659
2     1862
Name: count, dtype: int64

## <font color=#FFD700> Popülerliğe Dayalı - Popularity Based

In [15]:
product_grouped = df.groupby('productId').agg({'Rating': [np.size, np.sum, np.mean]})
# TR = ratings veri setini 'title' sütununa göre gruplayıp, her film için 'rating' sütununda 3 farklı istatistiği hesaplıyoruz:
#     np.size ile kaç değerlendirme yapıldığını (sayısı),
#     np.sum ile toplam değerlendirme puanını,
#     np.mean ile ortalama değerlendirme puanını hesaplıyoruz.

# EN = We are grouping the ratings dataset by the 'title' column, and calculating 3 different statistics for the 'rating' column:
#     np.size to get the number of ratings (count),
#     np.sum to get the total rating score,
#     np.mean to get the average rating score.


In [16]:
product_grouped

Rating               
             size  sum      mean
productId                       
0528881469     18   48  2.666667
0594033926     10   44  4.400000
0594451647     12   51  4.250000
0594481813     28  118  4.214286
0594481902     12   52  4.333333
...           ...  ...       ...
B00004THPR     10   43  4.300000
B00004THQ0     19   67  3.526316
B00004THQ5     18   67  3.722222
B00004TIZS     13   48  3.692308
B00004TJ0L     18   81  4.500000

[737 rows x 3 columns]

In [17]:
populer_product = product_grouped.sort_values(('Rating', 'mean'), ascending=False)
# TR = movie_grouped veri setini 'rating' sütununun 'mean' (ortalama) değerine göre azalan sırayla sıralıyoruz.
#     Bu, en yüksek ortalama puana sahip filmleri en üste getirir.

# EN = We are sorting the movie_grouped dataset by the 'mean' (average) value of the 'rating' column in descending order.
#     This brings the movies with the highest average ratings to the top.


In [18]:
populer_product.head()

Rating         
             size sum mean
productId                 
B00000J3NE     17  85  5.0
B00004S54K      8  40  5.0
B00002EQBU      7  35  5.0
B00002NAXD     12  60  5.0
B00000JDFI     11  55  5.0

In [19]:
grouped_sum = product_grouped['Rating']['sum'].sum()
# TR = movie_grouped veri setindeki 'rating' sütununun 'sum' (toplam) değerlerini topluyoruz. 
#     Bu, tüm filmler için toplam değerlendirme puanlarını toplar.

# EN = We are summing the 'sum' values from the 'rating' column in the movie_grouped dataset.
#     This gives the total rating score across all movies.

In [20]:
populer_product['percentage'] = product_grouped['Rating']['sum'].div(grouped_sum) * 100
# TR = movie_grouped veri setindeki her bir filmin 'rating' sütunundaki 'sum' (toplam) değerini, grouped_sum (tüm filmlerin toplam değerlendirme puanı) ile bölüyoruz.
#     Sonucu 100 ile çarparak, her filmin toplam değerlendirme puanının tüm filmler arasındaki yüzdesini hesaplıyoruz.

# EN = We are dividing the 'sum' (total) value from the 'rating' column in the movie_grouped dataset by grouped_sum (the total rating score for all movies).
#     We multiply the result by 100 to calculate the percentage of each movie's total rating score relative to the total across all movies.


In [21]:
populer_product.head()

Rating          percentage
             size sum mean           
productId                            
B00000J3NE     17  85  5.0   0.061574
B00004S54K      8  40  5.0   0.028976
B00002EQBU      7  35  5.0   0.025354
B00002NAXD     12  60  5.0   0.043464
B00000JDFI     11  55  5.0   0.039842

In [22]:
populer_product = populer_product.sort_values('percentage', ascending=False)
# TR = populer_movies veri setini 'percentage' sütununa göre azalan sırayla sıralıyoruz.
#     Bu, en yüksek yüzdelik değere sahip filmleri en üste getirir.
# EN = We are sorting the populer_movies dataset by the 'percentage' column in descending order.
#     This brings the movies with the highest percentage values to the top.

In [23]:
populer_product

Rating                 percentage
             size   sum      mean           
productId                                   
B00001P4ZH   1682  7486  4.450654   5.422869
B00001WRSJ   1280  5897  4.607031   4.271795
B00004T8R2   1379  5880  4.263959   4.259481
0972683275    824  3701  4.491505   2.681010
B00004SABB    796  3200  4.020101   2.318085
...           ...   ...       ...        ...
0899336795      9    15  1.666667   0.010866
B00000JH72     10    15  1.500000   0.010866
9269807207      8    14  1.750000   0.010142
B00000J4DT      5     8  1.600000   0.005795
B00000JII9      5     6  1.200000   0.004346

[737 rows x 4 columns]

In [24]:
populer_product['Rank'] = populer_product['percentage'].rank(ascending=False)
# TR = populer_movies veri setindeki 'percentage' sütununa göre her filme bir sıralama (rank) numarası veriyoruz.
#     Bu sıralama, yüzdelik değerlere göre azalan sırayla yapılır, yani en yüksek yüzdelik değere sahip film birinci sıradadır.

# EN = We are assigning a rank to each movie in the populer_movies dataset based on the 'percentage' column.
#     The ranking is done in descending order, meaning the movie with the highest percentage gets the top rank.

In [25]:
populer_product.head()

Rating                 percentage Rank
             size   sum      mean                
productId                                        
B00001P4ZH   1682  7486  4.450654   5.422869  1.0
B00001WRSJ   1280  5897  4.607031   4.271795  2.0
B00004T8R2   1379  5880  4.263959   4.259481  3.0
0972683275    824  3701  4.491505   2.681010  4.0
B00004SABB    796  3200  4.020101   2.318085  5.0

## <font color='#0F52BA'> Öznitelik Mühendisliği - Feature Engineering

### <font color=#007fff> Model - Modelling

In [26]:
df.head(1)

,userId,productId,Rating
0,AOPE42H34R0EC,B00000DM9W,5


In [27]:
x = df.drop('Rating', axis=1)
y = df['Rating']

In [28]:
x=pd.get_dummies(x,drop_first=True)
# Tr =  kategorik değişkenlerin sayısal değişkenlere dönüştürülmesi için kullanılır.
# En = It is used to convert categorical variables into numerical variables.

In [29]:
x.fillna(0, inplace=True)
y.fillna(0, inplace=True)

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
# TR = Veriyi eğitim ve test seti olarak ayırır; %20 test seti, %80 eğitim seti olacak şekilde bölünür
# EN = Splits the data into training and test sets; 20% for the test set and 80% for the training set

In [31]:
scaler = StandardScaler()
# TR = Avantajları:
# TR = Negatif ve pozitif değerler içeren verilerde performansı artırabilir.
# TR = Aykırı değerlere karşı daha dayanıklıdır.
# TR = Dezavantajları:
# TR = Veriyi kesin bir aralığa sığdırmaz (örneğin, 0 ile 1 arasında).

# EN = Advantages:
# EN = Can improve performance for data containing both negative and positive values.
# EN = More robust to outliers.
# EN = Disadvantages:
# EN = Does not fit the data into a specific range (e.g., between 0 and 1).

x_train = scaler.fit_transform(x_train)
# TR = Verileri ölçekleyerek, modelin daha hızlı ve etkili öğrenmesini sağlamak için tüm özellikleri aynı aralığa getiriyoruz.
# EN = By scaling the data, we bring all the features into the same range to allow the model to learn faster and more effectively.

x_test = scaler.transform(x_test)
# TR = Test verilerini eğitimde kullanılan aynı ölçekle dönüştürerek modelin test verileri üzerinde doğru tahminler yapmasını sağlar.
# EN = It enables the model to make accurate predictions on the test data by transforming the test data to the same scale used in training.

MemoryError: Unable to allocate 6.71 GiB for an array with shape (33119, 27180) and data type float64

In [ ]:
model = Sequential()
# TR = Modelin katmanlarını sırayla eklemek için Sequential kullanıyoruz.
# EN = We use Sequential to add layers of the model in order.

model.add(Dense(128, input_dim=x_train.shape[1]))
# TR = 128 nöronlu bir katman ekliyoruz.
# TR = input_dim=x_train.shape[1] Modelin ilk katmanına giriş verilerinin boyutunu tanımlayarak her özelliğin doğru şekilde işlenmesini sağlar.

# EN = We add a layer with 128 neurons.
# EN = input_dim=x_train.shape[1] Defines the size of the input data in the first layer of the model, ensuring that each feature is processed correctly.

model.add(Dense(64))
model.add(Dense(32))

model.add(Flatten())

model.add(Dense(1, activation='linear'))
# TR = Regresyon görevleri için lineer aktivasyon kullanıyoruz.
# EN = We use linear activation for regression tasks.

model.compile(loss='mse', optimizer='adam',metrics=['mean_squared_error'])
# TR = Modeli ortalama kare hata (mse) kaybı ile derliyoruz.
# EN = We compile the model with mean square error (mse) loss.

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# TR = EarlyStopping ekleyin: Eğitim sırasında model performansı iyileşmediğinde erken durması için kullanıyoruz.
# EN = Add EarlyStopping: We use it to stop early when model performance does not improve during training.

# TR = val_loss 10 epoch boyunca iyileşmezse eğitimi durduruyor ve en iyi ağırlıkları geri yüklüyor.
# EN = If val_loss does not improve for 10 epochs, it stops training and restores the best weights.

history=model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=100, callbacks=[early_stopping])
# TR = Modeli 100 epoch boyunca eğitiyoruz, fakat EarlyStopping ile durdurulabilir. Batch boyutu 128 olarak belirlenmiş.
# EN = We train the model for 100 epochs, but it can be stopped with EarlyStopping. Batch size is set to 128.

In [ ]:
model.summary()

In [ ]:
test_loss = model.evaluate(x_test, y_test)

# TR = İlk eleman kayıp değerini, ikinci eleman ise hesaplanan ek metriği içerir.
# EN = The first element contains the loss value, the second contains the additional metric.

print(f"Test kaybı: {test_loss[0]:.4f}")
print(f"Mean Squared Error: {test_loss[1]:.4f}")

In [ ]:
pred = model.predict(x_test)
# TR = modeli x_test ile predict özelliği ile tahmin ettik. predict=tahmin demek. Dahmin edip pred eşitledik.
# EN = We predicted the model with x_test and the predict feature. predict=means prediction. We guessed and equalized the pred

In [ ]:
r2_score(y_test, pred)
# TR = Bunu gerçek(y_test) değer ile tahmin(pred) edilen değerleri karşılaştır ve arasındaki farkı bul.
# EN = Compare this with the actual (y_test) value and the predicted (pred) values ​​and find the difference between them.

In [ ]:
mean_squared_error(y_test,pred)**.5
# TR = Burada, Root Mean Square Error bulduk. Bunu gerçek(y_test) değer ile tahmin(pred) edilen değerleri karşılaştır arasındaki farkı bul ve **.5 ile karekökünü al.
# EN = Here, we found Root Mean Square Error. Compare this with the actual (y_test) value and the predicted (pred) values, find the difference and take the square root of **.5.

In [ ]:
mean_absolute_error(y_test,pred)
# TR = mean_absolute_error ile (y_test,pred) kullanarak değerini hesaplama. 
# EN = Calculating the value of mean_absolute_error using (y_test,pred).

In [ ]:
residuals=y_test-pred.flatten()

In [ ]:
sns.kdeplot(x=residuals,fill=True);
# TR = Veri dağılımını pürüzsüz bir şekilde tahmin etmek için kullanılan bir yoğunluk grafiğidir. Kernel Yoğunluk Tahmini (KDE) ile verinin altında yatan olasılık dağılımını görselleştirir.
# EN = It is a density plot used to smoothly estimate the distribution of data. It visualizes the underlying probability distribution of data using Kernel Density Estimation (KDE).

In [ ]:
loss_f=pd.DataFrame(history.history)

In [ ]:
loss_f.plot();

In [ ]:
pickle.dump(model,open('Amazon.pkl','wb'))